In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=67905 sha256=6e4cf06974b10555cd7f83803916136e6078a1bb4a7dc87e9e6b633914bdf4fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ndu9rp1_/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.end2end import LFF

bias_model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
lff = LFF(
    trainset=trainset,
    bias_model=bias_model,
    debias_model=model,
    batch_size=64,
    bias_optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    debias_optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    q=1.0,
    num_epochs=5,
    device=device,
    verbose=True
)
lff.train()

Epoch 4: 100%|██████████| 751/751 [00:06<00:00, 119.97batch/s, bias_accuracy=25.0%, bias_loss=0.826, debias_accuracy=25.0%, debias_loss=0.805]      


In [5]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 0.0
Group (0, 1) Test Accuracy: 0.0
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 100.0
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 100.0
Group (1, 3) Test Accuracy: 100.0
Group (1, 4) Test Accuracy: 100.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 0.0
Group (2, 3) Test Accuracy: 0.0
Group (2, 4) Test Accuracy: 0.0
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 0.0
Group (3, 3) Test Accuracy: 0.0
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 0.0


{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 100.0,
 (1, 1): 100.0,
 (1, 2): 100.0,
 (1, 3): 100.0,
 (1, 4): 100.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.0}

In [11]:
evaluator.worst_group_accuracy

((0, 0), 0.0)

In [10]:
evaluator.average_accuracy

20.14832097325223

In [9]:
evaluator.evaluate_spurious_task()

20.01